# Uso del Middleware Integrado para Resumir una Conversación Larga

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model="gpt-4o-mini",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

from langchain.messages import HumanMessage, AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Are you ready to play the JFK QA game?"),
        AIMessage(content="Sure!"),
        HumanMessage(content="Who was the favorite sister or JFK?"),
        AIMessage(content="Her sister Kick."),
        HumanMessage(content="Correct! Who was his favorite brother?"),
        AIMessage(content="Hmmm, that is difficult. I would say Ted. He loved Bobby very much, but Bobby was very different from him."),
        HumanMessage(content="Correct! What was the main source of pain of JFK on a daily basis?"),
        AIMessage(content="Back pain."),
        HumanMessage(content="Correct again! Did JFK have dogs?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

print(response["messages"][-1].content)

#### Vale. Usemos pprint para ver la respuesta detallada

In [ ]:
from pprint import pprint

pprint(response)

#### Como podéis ver, la respuesta incluye un resumen de la conversación hasta la fecha. Vamos a imprimirlo.

In [ ]:
print(response["messages"][0].content)

## Vale. Ahora expliquemos el código anterior en términos sencillos

A continuación se muestra el mismo código, explicado **en términos simples, línea por línea**, además de una explicación clara de **qué hace `SummarizationMiddleware`**.

---

#### Qué está haciendo este programa (visión general)

Estás creando un **agente** de chat (un bucle de asistente inteligente) que:

1. Usa un LLM (`gpt-4o-mini`)
2. **Recuerda la conversación** usando un "checkpointer" (almacenamiento de memoria)
3. Usa **SummarizationMiddleware** para *resumir automáticamente el historial de chat antiguo* cuando se vuelve demasiado largo
4. Ejecuta el agente con una lista de mensajes de chat e imprime la última respuesta del agente

Resumen del middleware: es una forma de "interceptar/controlar" lo que sucede dentro del bucle del agente.

---

#### Importaciones

```python
from langchain.agents import create_agent
```

* Importa `create_agent`, un helper que construye un agente por ti (un agente = modelo + herramientas + memoria + comportamiento del bucle).

```python
from langgraph.checkpoint.memory import InMemorySaver
```

* Importa un **checkpointer en memoria**.
* Un *checkpointer* almacena el estado del agente para que el agente pueda reanudar una conversación más tarde (por hilo). Para demostraciones/prototipos rápidos, LangChain recomienda `InMemorySaver`.

```python
from langchain.agents.middleware import SummarizationMiddleware
```

* Importa el middleware que **resumirá automáticamente el historial de conversación** cuando se alcance algún umbral.

---

#### Crear el agente

```python
agent = create_agent(
```

* Comienza a construir un objeto agente.

```python
    model="gpt-4o-mini",
```

* Establece el modelo principal que el agente usará para responder.

```python
    checkpointer=InMemorySaver(),
```

* Añade "persistencia de memoria a corto plazo" usando un almacén en memoria.
* Esto permite que el agente mantenga un historial de conversación **por id de hilo**, para que no se mezclen múltiples conversaciones.

```python
    middleware=[
```

* Añade componentes middleware (piensa: "plugins" que se ejecutan en ciertos puntos dentro del bucle del agente).

```python
        SummarizationMiddleware(
```

* Activa la resumición automática de mensajes antiguos.

```python
            model="gpt-4o-mini",
```

* El modelo utilizado **para escribir el resumen**.
* (Puedes usar el mismo o un modelo más barato/rápido que el principal.)

```python
            trigger=("tokens", 100),
```

* **Cuándo resumir.**
* `("tokens", 100)` significa: *si el contexto de la conversación está a punto de superar ~100 tokens (según el contador de tokens), resume el contenido antiguo.*
* **Nota importante para principiantes:** 100 tokens es *muy poco* (como unos pocos mensajes cortos), por lo que esto resumirá de forma muy agresiva.

```python
            keep=("messages", 1)
```

* **Cuánto chat reciente mantener "tal cual"** después de resumir.
* `("messages", 1)` significa: mantener solo el **1 mensaje** más reciente sin cambios; el contenido más antiguo se comprime en un resumen.

```python
        )
    ],
)
```

* Termina de construir el agente.

---

#### Crear objetos de mensaje

```python
from langchain.messages import HumanMessage, AIMessage
```

* Importa tipos de mensajes.
* `HumanMessage` = texto del usuario, `AIMessage` = texto del asistente.

---

#### Invocar (ejecutar) el agente con una conversación

```python
response = agent.invoke(
```

* Ejecuta el agente una vez y devuelve un objeto de respuesta (que incluye mensajes/actualizaciones de estado).

```python
    {"messages": [
```

* La entrada es un diccionario con una lista de `messages`.
* Estás dando al agente una conversación "hasta ahora".

```python
        HumanMessage(content="Are you ready to play the JFK QA game?"),
        AIMessage(content="Sure!"),
        HumanMessage(content="Who was the favorite sister or JFK?"),
        AIMessage(content="Her sister Kick."),
        HumanMessage(content="Correct! Who was his favorite brother?"),
        AIMessage(content="Hmmm, that is difficult. I would say Ted. He loved Bobby very much, but Bobby was very different from him."),
        HumanMessage(content="Correct! What was the main source of pain of JFK on a daily basis?"),
        AIMessage(content="Back pain."),
        HumanMessage(content="Correct again! Did JFK have dogs?"),
```

* Esta es una conversación de ida y vuelta.
* El último mensaje es una pregunta del usuario: "Did JFK have dogs?"
* El agente debería responder a esa última pregunta.

```python
        ]},
```

* Termina la lista de mensajes y el diccionario de entrada.

```python
    {"configurable": {"thread_id": "1"}}
```

* Esta es la **configuración** para la ejecución.
* `thread_id="1"` le dice al checkpointer: "almacena/recupera memoria para el hilo de conversación #1".
* Así es como se mantienen sesiones de chat separadas.

```python
)
```

* Termina la llamada al agente.

---

#### Imprimir la respuesta final del agente

```python
print(response["messages"][-1].content)
```

* `response["messages"]` es la lista de mensajes actualizada después de que el agente respondiera.
* `[-1]` significa "el último mensaje".
* `.content` obtiene el texto.
* Así que esto imprime la respuesta más reciente del agente.

---

#### Qué hace `SummarizationMiddleware` (explicación simple)

`SummarizationMiddleware` es un **compresor automático de historial de chat**:

* **Monitoriza cuán grande es tu historial de mensajes** (por tokens, número de mensajes o fracción de contexto).
* Cuando se alcanza el **umbral de disparo** (en tu código: 100 tokens):

  1. Toma la parte *más antigua* de la conversación
  2. Llama a un modelo (en tu código: `gpt-4o-mini`) para **resumir** esa parte antigua
  3. Reemplaza esa parte antigua con un **resumen corto**
  4. Mantiene algunos mensajes más recientes intactos según `keep` (en tu código: mantener solo 1 mensaje reciente)

Un detalle de la documentación de referencia: "mantiene la continuidad del contexto asegurando que **los pares de mensajes AI/Tool permanezcan juntos**" (para que no se rompa el significado de las interacciones con herramientas).

#### En la configuración específica que usamos

* `trigger=("tokens", 100)` → resumir *muy rápidamente*
* `keep=("messages", 1)` → mantener casi nada textual, solo el/los último(s) mensaje(s)
* Resultado: el agente a menudo verá algo como:

  * **Resumen:** "Estamos jugando un juego de preguntas y respuestas sobre JFK. El usuario preguntó X, el asistente respondió Y…"
  * **Mensaje más reciente:** "Correct again! Did JFK have dogs?"

## Cómo ejecutar este código desde Visual Studio Code
* Abre el Terminal.
* Asegúrate de estar en la carpeta del proyecto.
* Asegúrate de tener el entorno poetry activado.
* Introduce y ejecuta el siguiente comando:
    * `python 011-mid-to-summ-conversation.py`